In [1]:
import os
import json

from library_dicom.dicom_processor.tools.folders import *
from library_dicom.dicom_processor.tools.series import get_series_object

import csv
from library_dicom.dicom_processor.model.Series import Series

In [5]:
series_paths = get_series_path("/media/salim/DD 2To/FLIP/DICOM")
export_folder = '/media/salim/DD 2To/FLIP/FLIP_JSON'

In [ ]:
#Number of serie 
len(series_paths)

In [6]:
index_problem = []
for serie_path in series_paths:
    #print(series_paths.index(serie_path))
    try:
        dicom_serie = get_series_object(serie_path)
        dicomsInfo = dicom_serie.get_series_details()
        write_json_file(export_folder, dicomsInfo['series']['SeriesInstanceUID'], dicomsInfo)
    except Exception as err:
        print(err)
        print(dicomsInfo)
        index_problem.append(series_paths.index(serie_path))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [3]:
#Load CSV result pet0/2/4

csv_pet0_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/petIni.csv'

with open(csv_pet0_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ';') #liste pour chaque ligne 
    csv_data = []
    for row in reader :
        csv_data.append(row)
        
del csv_data[0] #enlever première ligne


In [6]:
#list of json from serie in Validated_DICOM
json_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/RELEVANCE_json'
list_json = os.listdir(json_directory)
number_of_json = len(list_json)
print(number_of_json)

455


In [8]:
#get study_uid of every serie in Validated_DICOM in order to find serie which belong to the same study
study_uid = []
for json_ in list_json :
    #print(list_json.index(json_))
    with open(os.path.join(json_directory, json_)) as json_file :
        reader = json.load(json_file)
        data = []
        for info in reader['study']['StudyInstanceUID'] :
            data.append(info)
            study_uid_json = "".join(data)

    if study_uid_json not in study_uid : 
        study_uid.append(study_uid_json)

In [49]:
#get list [[json_1, json_2, patient_id, study_uid], [...]] of all dataset for every study , from a CSV 

dataset = []

for item in csv_data :
    
    patient_id = item[1]
    study_uid = item[2]
    #csv_file = item[]
    liste = []

    for number_json in range(number_of_json) : 
        with open(os.path.join(json_directory, list_json[number_json])) as json_file :
            reader = json.load(json_file)
            data = []
        
            for info in reader['study']['StudyInstanceUID'] : 
                data.append(info)
                study_uid_json = "".join(data)

                if study_uid == study_uid_json : 
                    liste.append(os.path.join(json_directory, list_json[number_json]))
                    
    liste.append(csv_file)                
    liste.append(patient_id)
    liste.append(study_uid)
    

    dataset.append(liste)


In [11]:
#get list [[json_1, json_2, patient_id, study_uid], [...]] of all dataset for every study , without CSV 

dataset = []

for uid in study_uid :
    liste = []
    for json_ in list_json :
        with open(os.path.join(json_directory, json_)) as json_file :
            reader = json.load(json_file)
            data = []
            data_2 = []
            for info in reader['study']['StudyInstanceUID'] :
                data.append(info)
                study_uid_json = "".join(data)
            for info in reader['patient']['PatientID'] : 
                data_2.append(info)
                patient_id = "".join(data_2)

            if uid == study_uid_json : 
                    liste.append(os.path.join(json_directory, json_))

    liste.append(patient_id)
    liste.append(uid)
    

    dataset.append(liste)


In [ ]:
#Clean the dataset : if not 2 json, if same modality, if double study_uid

In [17]:
#If no 2 JSON in the study :
no_json = []
for liste in dataset : 
    if len(liste) != 4 : #or 5 if CSV in it 
        no_json.append(liste)

print(len(no_json))

for serie in no_json : 
    dataset.remove(serie)

print(len(dataset))

67
150


In [18]:
#If double study_uid, so double liste : 
complete_liste = []
complete_liste.append(dataset[0])
for i in range(1, len(dataset)) :
    if dataset[i] not in complete_liste : 
        complete_liste.append(dataset[i])

dataset = complete_liste
print(len(dataset))

150


In [19]:
#Check if there is a PET and a CT scan. If two CT or two PET, delete from dataset
modality = []

for liste in dataset : 
    subliste = []
    for i in range(2) : 
        with open(liste[i]) as json_file : 
            data = []
            reader = json.load(json_file)
            for info in reader['series']['Modality'] :
                data.append(info)
                mod = "".join(data)
            subliste.append(mod)
    modality.append(subliste)


index_false_modality = []
for modal in modality : 
    index = modality.index(modal)
    if modal[0] == modal[1] :
        index_false_modality.append(index)

print(index_false_modality)

[106]


In [24]:
#delete serie with 2 CT or 2 PET in dataset 
for index in index_false_modality : 
    study = dataset[index]
    modality_study = modality[index]
    
    dataset.remove(study)
    modality.remove(modality_study)

In [26]:
#Now, can prepare json file to generate nifti in "nifti_builder_from_json.ipynb"

In [31]:
#rewrite dataset with modality in it 

study_results = []

for liste in dataset:
    subliste = []
    for number_serie in range(2) : 
        with open(liste[number_serie]) as json_file : 
            data = []
            reader = json.load(json_file)
            for info in reader['path']:
                data.append(info)
                path = "".join(data)

            subliste.append(path) #add path of the serie 
            subliste.append(modality[complete_liste.index(liste)][number_serie]) #add modality of the serie 
        
    subliste.append(liste[-1]) #add study_uid of the serie 
    subliste.append(liste[-2]) #add patient_id of the serie 
    #subliste.append(liste[-3]) #add csv_file of the serie if it exists
    study_results.append(subliste) 

print(len(study_results))

149


In [30]:
study_results

oi6/12011101141103/PET_Screening/1.3.46.670589.28.2.7.2200166146.2.20584.0.1394203512',
  'PT',
  '/media/deeplearning/Elements/RElevance envoi 2/Envoi6/12011101141103/PET_Screening/1.2.840.113704.1.111.5632.1394199118.8',
  'CT',
  '1.2.840.113704.1.111.1600.1394198953.13',
  '12011101191104'],
 ['/media/deeplearning/Elements/RElevance envoi 2/Envoi9/12011101231104/PET_Screening/1.3.12.2.1107.5.1.4.1003.30000013020808280207800010581',
  'PT',
  '/media/deeplearning/Elements/RElevance envoi 2/Envoi9/12011101231104/PET_Screening/1.3.12.2.1107.5.1.4.1003.30000013020808274609300007805',
  'CT',
  '1.2.928.1024109150',
  '12011101191104'],
 ['/media/deeplearning/Elements/RElevance envoi 2/Envoi18/62011201291111/PET_24weeks/1.3.46.670589.28.2.12.30.26442.56633.2.2792.0.1427878226',
  'PT',
  '/media/deeplearning/Elements/RElevance envoi 2/Envoi18/62011201291111/PET_24weeks/1.2.840.113704.1.111.5832.1427789993.8',
  'CT',
  '1.2.826.0.1.3680043.2.1174.4.100811870731746',
  '12011101191104']]

In [65]:
#save results as json 
directory = ''
filename = ''
write_json_file(directory, filename, study_results)

In [ ]:
#if no csv, create csv_file to save patient_id, study_uid of dataset 
nifti_directory = ''
filename = ''
with open(os.path.join(nifti_directory, filename), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["PATIENT ID", "STUDY UID"])
    for serie in new_liste: 
         csv_writer.writerow([serie[-1], serie[-2]])